In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, confusion_matrix
from sklearn.model_selection import KFold


Using matplotlib backend: TkAgg


In [8]:
# importation de donnees 
df = pd.read_csv("database_TT_Churn.csv")
df 

df.head()

,id_client,genre,age,marie,num_tel,nb_jours_abonne,duree_appel_jour,nb_appel_jour,cout_appel_jour,duree_appel_soiree,...,nb_appel_nuit,cout_appel_nuit,duree_appel_inter,nb_appel_inter,cout_appel_inter,active_msg_vocaux,nb_msg_vocaux,nb_reclamation,churn,offer_type
0,382-4657,femme,37,yes,98505453,128.0,265.1,110.0,45.07,197.4,...,91.0,11.01,10.0,3.0,2.70,yes,25,1.0,False,Hayya
1,371-7191,homme,46,no,97321658,107.0,161.6,123.0,27.47,195.5,...,103.0,11.45,13.7,3.0,3.70,yes,26,1.0,False,PRE - 1=11
2,358-1921,homme,50,no,98653270,137.0,243.4,114.0,41.38,121.2,...,104.0,7.32,12.2,5.0,3.29,no,0,0.0,False,PRE - 900 bonus
3,375-9999,homme,78,yes,96303256,84.0,299.4,71.0,50.90,61.9,...,89.0,8.86,6.6,7.0,1.78,no,0,2.0,False,PRE - AHLA
4,330-6626,femme,75,yes,96412387,75.0,166.7,113.0,28.34,148.3,...,121.0,8.41,10.1,3.0,2.73,no,0,3.0,False,PRE - Binetna


In [9]:
df.shape

(5000, 23)

In [10]:
# information sur l'ensemble de donnees
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id_client           5000 non-null   object 
 1   genre               5000 non-null   object 
 2   age                 5000 non-null   int64  
 3   marie               5000 non-null   object 
 4   num_tel             5000 non-null   int64  
 5   nb_jours_abonne     4990 non-null   float64
 6   duree_appel_jour    5000 non-null   float64
 7   nb_appel_jour       4993 non-null   float64
 8   cout_appel_jour     5000 non-null   float64
 9   duree_appel_soiree  5000 non-null   float64
 10  nb_appel_soiree     4994 non-null   float64
 11  cout_appel_soiree   5000 non-null   float64
 12  duree_appel_nuit    5000 non-null   float64
 13  nb_appel_nuit       4989 non-null   float64
 14  cout_appel_nuit     5000 non-null   float64
 15  duree_appel_inter   5000 non-null   float64
 16  nb_app

In [11]:
df.describe().T


,count,mean,std,min,25%,50%,75%,max
age,5000.0,4.650680e+01,16.743513,19.0,3.200000e+01,46.00,60.00,80.00
num_tel,5000.0,9.844445e+07,371630.462063,90032100.0,9.850545e+07,98505453.00,98505453.00,99652014.00
nb_jours_abonne,4990.0,1.002707e+02,39.704130,1.0,7.300000e+01,100.00,127.00,243.00
duree_appel_jour,5000.0,1.802889e+02,53.894699,0.0,1.437000e+02,180.10,216.20,351.50
nb_appel_jour,4993.0,1.000190e+02,19.828023,0.0,8.700000e+01,100.00,113.00,165.00
cout_appel_jour,5000.0,3.064967e+01,9.162069,0.0,2.443000e+01,30.62,36.75,59.76
duree_appel_soiree,5000.0,2.006366e+02,50.551309,0.0,1.663750e+02,201.00,234.10,363.70
nb_appel_soiree,4994.0,1.002147e+02,19.820534,0.0,8.700000e+01,100.00,114.00,170.00
cout_appel_soiree,5000.0,1.705432e+01,4.296843,0.0,1.414000e+01,17.09,19.90,30.91
duree_appel_nuit,5000.0,2.003916e+02,50.527789,0.0,1.669000e+02,200.40,234.70,395.00


In [12]:
# nombre de valeurs uniques par colonne 
df.nunique()

id_client             5000
genre                    2
age                     62
marie                    2
num_tel                 33
nb_jours_abonne        218
duree_appel_jour      1961
nb_appel_jour          123
cout_appel_jour       1961
duree_appel_soiree    1879
nb_appel_soiree        126
cout_appel_soiree     1659
duree_appel_nuit      1853
nb_appel_nuit          131
cout_appel_nuit       1028
duree_appel_inter      170
nb_appel_inter          21
cout_appel_inter       170
active_msg_vocaux        2
nb_msg_vocaux           48
nb_reclamation          10
churn                    2
offer_type              40
dtype: int64

In [13]:
#ANALYSE UNIVARIEE

In [14]:
# varaiables catregorielles
categorical_columns = ['genre', 'marie', 'offer_type','active_msg_vocaux', 'churn']
categorical_columns

['genre', 'marie', 'offer_type', 'active_msg_vocaux', 'churn']

In [15]:
# creation d une fonction de construction de histogrammes a barres et de maniere interactive
def plot_distribution(column):
    
    if column == 'churn':
        # Plot donut chart for 'churn' column
        plt.figure(figsize=(8, 6))
        df[column].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['#ff9999','#66b3ff'], startangle=90, wedgeprops=dict(width=0.3), labels=None)
        centre_circle = plt.Circle((0,0),0.70,fc='white')
        fig = plt.gcf()
        fig.gca().add_artist(centre_circle)
        plt.axis('equal')  
        plt.title('Churn Distribution')
        plt.legend(labels=df[column].value_counts().index, loc='best')
        plt.show()
    else:
        # Plot bar chart for other columns
        plt.figure(figsize=(8, 6))
        ax = sns.countplot(x=column, data=df, palette='coolwarm')
        plt.ylabel('Nombre de Clients')
        plt.title('Distribution de ' + column)
        plt.xticks(rotation=45)
        
        # Add percentage labels to the bars for columns other than 'offer_type'
        if column != 'offer_type':
            total = len(df[column])
            for p in ax.patches:
                percentage = '{:.1f}%'.format(100 * p.get_height() / total)
                x = p.get_x() + p.get_width() / 2 - 0.05
                y = p.get_height()
                ax.annotate(percentage, (x, y), ha='center', va='bottom')
        
        plt.show()

# Now you can use interact
interact(plot_distribution, column=categorical_columns)


interactive(children=(Dropdown(description='column', options=('genre', 'marie', 'offer_type', 'active_msg_voca…

<function __main__.plot_distribution(column)>

In [16]:
# varaiables quantitatives
numerical_columns = ['age','nb_jours_abonne','duree_appel_jour','nb_appel_jour','cout_appel_jour','duree_appel_soiree','nb_appel_soiree',
                    'cout_appel_soiree','duree_appel_nuit','nb_appel_nuit','cout_appel_nuit','duree_appel_inter','nb_appel_inter',
                    'cout_appel_inter','nb_msg_vocaux','nb_reclamation']
numerical_columns

['age',
 'nb_jours_abonne',
 'duree_appel_jour',
 'nb_appel_jour',
 'cout_appel_jour',
 'duree_appel_soiree',
 'nb_appel_soiree',
 'cout_appel_soiree',
 'duree_appel_nuit',
 'nb_appel_nuit',
 'cout_appel_nuit',
 'duree_appel_inter',
 'nb_appel_inter',
 'cout_appel_inter',
 'nb_msg_vocaux',
 'nb_reclamation']

In [17]:
# Distribution d'age
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, 120)]

# Create a new column in the DataFrame to represent age intervals
df['age_interval'] = pd.cut(df['age'], bins=[interval[0] for interval in age_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.countplot(x='age_interval', data=df,  color='#FFB6C1',order=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])
plt.title("Distribution d'âge")
plt.xlabel('Age')
plt.ylabel('Nombre de clients')
plt.legend( loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [18]:
#Distribution de Nombre d'Appel pour chaque type d'appel




# Total number of calls for each type of call
appel_jour = df['nb_appel_jour'].sum()
appel_soiree = df['nb_appel_soiree'].sum()
appel_nuit = df['nb_appel_nuit'].sum()
appel_inter = df['duree_appel_inter'].sum()

# Labels for each type of call
labels = ['Appels de Jour', 'Appels de Soirée', 'Appels de Nuit', 'Appels Internationaux']

x = np.arange(len(labels))
width = 0.35

# Creating the bar plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("Distribution du Nombre d'Appel pour chaque type d'appel")
bars = ax.bar(x, [appel_jour, appel_soiree, appel_nuit, appel_inter], width, color='#F4CCCC')

# Adding labels, title, and legend
ax.set_xlabel("Types d'appel")
ax.set_ylabel('Nombre des appels')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45)
ax.legend(loc='upper right')

# Displaying the plot
plt.tight_layout()
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [19]:
#Distribution de la Duration des Appels pour chaque type d'appel

# Assuming you have already loaded your data into a DataFrame called df

# Select the columns containing the durations for each time period
jour_duration = df['duree_appel_jour']
soiree_duration = df['duree_appel_soiree']
nuit_duration = df['duree_appel_nuit']
inter_duration = df['duree_appel_inter']
# Define pastel colors
colors = [(1, 0.75, 0.8), (0.75, 0.85, 1), (0.85, 1, 0.85),(0.7, 0.6, 0.8)]  # Pastel shades of pink, blue, and green

# Create subplots for each time period
plt.figure(figsize=(10, 6))

# Plot the distribution of durations for each time period without bars
sns.distplot(jour_duration, hist=False, color=colors[0], label='Jour')
sns.distplot(soiree_duration, hist=False, color=colors[1], label='Soiree')
sns.distplot(nuit_duration, hist=False, color=colors[2], label='Nuit')
sns.distplot(inter_duration, hist=False, color=colors[3], label='Internationale')

# Add labels and title
plt.xlabel('Duration')
plt.ylabel('Density')
plt.title('Distribution of Call Durations by Time Period')
plt.legend()

# Show the plot
plt.show()


C:\Users\chino\AppData\Local\Temp\ipykernel_15644\2556210571.py:17: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(jour_duration, hist=False, color=colors[0], label='Jour')
C:\Users\chino\AppData\Local\Temp\ipykernel_15644\2556210571.py:18: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `kdeplot` (an axes-level function for kernel density plots).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.d

In [20]:


#Distribution du Cout des Appels pour chaque type d'appel
count_data = pd.DataFrame({
    'Time_Period': ['Jour', 'Soiree', 'Nuit','Internationaux'],
    'Count': [df['cout_appel_jour'].sum(), df['cout_appel_soiree'].sum(), df['cout_appel_nuit'].sum(),df['cout_appel_inter'].sum()]
})

# Define pastel colors
colors = [(0.6, 0.4, 0.8)]  # Shades of pink, blue, and green

# Create grouped bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Time_Period', y='Count', data=count_data, palette=colors)

# Add labels and title
plt.xlabel("Types d'appels")
plt.ylabel("Nombre d'Appels")
plt.title("Distribution du Cout des Appels pour chaque type d'appel")

# Show the plot
plt.show()




C:\Users\chino\AppData\Local\Temp\ipykernel_15644\1202556344.py:12: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='Time_Period', y='Count', data=count_data, palette=colors)
C:\Users\chino\AppData\Local\Temp\ipykernel_15644\1202556344.py:12: UserWarning: 
The palette list has fewer values (1) than needed (4) and will cycle, which may produce an uninterpretable plot.
  sns.barplot(x='Time_Period', y='Count', data=count_data, palette=colors)


In [21]:
#  Distribution du nombre des réclamations
plt.figure(figsize=(8,8))
sns.countplot(x=df['nb_reclamation'], palette=['#FFC1CC','#FF66CC','#FF91A4'])
total = len(df['nb_reclamation'])
plt.xlabel("Nombre de réclamations ")
plt.ylabel("Nombre de clients")
plt.title("Distribution du nombre des réclamations")


C:\Users\chino\AppData\Local\Temp\ipykernel_15644\2860257450.py:3: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x=df['nb_reclamation'], palette=['#FFC1CC','#FF66CC','#FF91A4'])
C:\Users\chino\AppData\Local\Temp\ipykernel_15644\2860257450.py:3: UserWarning: 
The palette list has fewer values (3) than needed (10) and will cycle, which may produce an uninterpretable plot.
  sns.countplot(x=df['nb_reclamation'], palette=['#FFC1CC','#FF66CC','#FF91A4'])


Text(0.5, 1.0, 'Distribution du nombre des réclamations')

In [22]:
# creation d'une fonction de constsruction de boites a moustaches et de maniere interactive 
def box_plot(column):
    sns.boxplot(y =df[column])
    plt.title('Boxplot of '+str(column))
    return plt.show()
#interact
interact(box_plot, column= numerical_columns)

interactive(children=(Dropdown(description='column', options=('age', 'nb_jours_abonne', 'duree_appel_jour', 'n…

<function __main__.box_plot(column)>

In [23]:
#ANALYSE BIVARIEE

In [24]:
#genre vs churn

In [25]:
#nb_reclamation vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='nb_reclamation', hue='churn', data=df)
plt.title('Répartition du churn selon nombre de réclamations')
plt.xlabel('Nombre de réclamation')
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.show()

In [26]:
sns.boxplot(x='churn', y='nb_reclamation', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel('Nombre de réclamation')
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\2762798356.py:1: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='nb_reclamation', data=df, palette=['blue', 'orange'])


In [27]:
#etat_civile vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='marie', hue='churn', data=df)
plt.title("Répartition du churn selon l'état civile")
plt.xlabel('marié')
plt.ylabel('Nombre de clients')
plt.legend(title='churn', loc='upper right')
plt.show()

In [28]:
#age vs churn

# Define age intervals
age_intervals = [(0, 18), (18, 30), (30, 50), (50, 70), (70, 120)]

# Create a new column in the DataFrame to represent age intervals
df['age_interval'] = pd.cut(df['age'], bins=[interval[0] for interval in age_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.countplot(x='age_interval', hue='churn', data=df, order=[f'{interval[0]}-{interval[1]}' for interval in age_intervals])
plt.title("Répartition du churn selon l'age")
plt.xlabel('Age')
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [29]:
sns.boxplot(x='churn', y='age', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel('Age ')
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\1958056296.py:1: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='age', data=df, palette=['blue', 'orange'])


In [30]:
#nb_jours_abonne vs churn

# Define subscription days intervals
abonnement_intervals = [(0, 50), (50, 100), (100, 150), (150, 200), (200, 250)]

# Create a new column in the DataFrame to represent subscription days intervals
df['subscription_interval'] = pd.cut(df['nb_jours_abonne'], bins=[interval[0] for interval in abonnement_intervals] + [np.inf], labels=[f'{interval[0]}-{interval[1]}' for interval in abonnement_intervals])

# Plot the bar chart using subscription_interval
plt.figure(figsize=(10, 6))
sns.countplot(x='subscription_interval', hue='churn', data=df, order=[f'{interval[0]}-{interval[1]}' for interval in abonnement_intervals])
plt.title("Répartition du churn selon la période d'abonnement")
plt.xlabel("Période d'abonnement en jours")
plt.ylabel('Nombre de clients')
plt.legend(title='Churn', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [31]:
sns.boxplot(x='churn', y='nb_jours_abonne', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Période d'abonnement en jours")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\1100228588.py:1: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='nb_jours_abonne', data=df, palette=['blue', 'orange'])


In [32]:
#active_msg_vocaux vs churn


plt.figure(figsize=(8, 6))
sns.countplot(x='active_msg_vocaux', hue='churn', data=df)
plt.title(" Répartition du churn selon l’activation des messages vocaux")
plt.xlabel('Active messages vocaux')
plt.ylabel('Nombre de clients')
plt.legend(title='churn', loc='upper right')
plt.show()

In [33]:
# Calculate the percentage distribution of churn based on the activation of voicemail messages
voicemail_churn_distribution = df.groupby('active_msg_vocaux')['churn'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()

# Pivot the table to make it easier to read
voicemail_churn_pivot = voicemail_churn_distribution.pivot(index='active_msg_vocaux', columns='churn', values='percentage')

# Rename the columns for better clarity
voicemail_churn_pivot.columns = ['Not Churned (%)', 'Churned (%)']

# Display the table
print(voicemail_churn_pivot)


                   Not Churned (%)  Churned (%)
active_msg_vocaux                              
no                       83.556040    16.443960
yes                      92.278577     7.721423


In [34]:
#nb_msg_vocaux vs churn 
sns.boxplot(x='churn', y='nb_msg_vocaux', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Nombre de message vocaux")
plt.title(" Répartition du churn selon nombre de message vocaux")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\695860232.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='nb_msg_vocaux', data=df, palette=['blue', 'orange'])


In [35]:
#cout_appel_jour vs churn
sns.boxplot(x='churn', y='cout_appel_jour', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Jour")
plt.title(" Répartition du churn selon le cout d'appel par jour")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\2676615220.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_jour', data=df, palette=['blue', 'orange'])


In [36]:
#cout_appel_soiree vs churn
sns.boxplot(x='churn', y='cout_appel_soiree', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Soirée")
plt.title("Répartition du churn selon le cout d'appel par soirée")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\28414109.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_soiree', data=df, palette=['blue', 'orange'])


In [37]:
#cout_appel_nuit vs churn
sns.boxplot(x='churn', y='cout_appel_nuit', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("Nuit")
plt.title("Répartition du churn selon le cout d'appel par nuit")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\1306726788.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_nuit', data=df, palette=['blue', 'orange'])


In [38]:
#cout_appel_intern vs churn
sns.boxplot(x='churn', y='cout_appel_inter', data=df, palette=['blue', 'orange'])
plt.xlabel('Churn')
plt.ylabel("International")
plt.title("Répartition du churn selon le cout d'appels internationaux")
plt.show()

C:\Users\chino\AppData\Local\Temp\ipykernel_15644\1803368150.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='churn', y='cout_appel_inter', data=df, palette=['blue', 'orange'])


In [39]:
df.isna().sum()

id_client                   0
genre                       0
age                         0
marie                       0
num_tel                     0
nb_jours_abonne            10
duree_appel_jour            0
nb_appel_jour               7
cout_appel_jour             0
duree_appel_soiree          0
nb_appel_soiree             6
cout_appel_soiree           0
duree_appel_nuit            0
nb_appel_nuit              11
cout_appel_nuit             0
duree_appel_inter           0
nb_appel_inter             12
cout_appel_inter            0
active_msg_vocaux           8
nb_msg_vocaux               0
nb_reclamation             12
churn                       4
offer_type               4000
age_interval                0
subscription_interval      10
dtype: int64

In [40]:
print(df.duplicated().sum())

0


In [41]:
#MODELING PHASE

In [42]:
data = pd.read_csv('encoded_TT_Churn.csv')
data

,age,marie,genre,nb_jours_abonne,active_msg_vocaux,nb_msg_vocaux,nb_appel_jour,duree_appel_jour,cout_appel_jour,nb_appel_soiree,...,cout_appel_soiree,nb_appel_nuit,duree_appel_nuit,cout_appel_nuit,nb_appel_inter,duree_appel_inter,cout_appel_inter,nb_reclamation,churn,offer_type_target_mean
0,19,0,0,34,0,0,80,128.8,21.90,93,...,17.74,103,202.1,9.09,7.0,14.0,3.78,1,0,0.144750
1,19,0,0,38,0,0,110,96.9,16.47,75,...,20.52,69,141.0,6.35,3.0,10.8,2.92,0,0,0.144750
2,19,0,0,43,1,36,68,93.1,15.83,83,...,20.92,109,94.9,4.27,8.0,8.7,2.35,1,0,0.144750
3,19,0,0,48,1,37,81,138.3,23.51,74,...,15.95,113,158.0,7.11,1.0,11.6,3.13,2,0,0.144750
4,19,0,0,60,0,0,77,207.3,35.24,105,...,17.67,89,108.2,4.87,5.0,12.9,3.48,1,0,0.144750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,80,1,1,71,0,0,130,123.8,21.05,91,...,19.25,96,213.9,9.63,7.0,8.8,2.38,2,0,0.144750
4996,80,1,1,82,0,0,112,172.4,29.31,89,...,11.19,78,182.8,8.23,9.0,8.8,2.38,1,0,0.144750
4997,80,1,1,137,0,0,107,75.8,12.89,98,...,21.84,118,204.4,9.20,3.0,10.5,2.84,2,0,0.144750
4998,80,1,1,139,1,23,129,157.6,26.79,96,...,21.00,112,259.2,11.66,2.0,13.7,3.70,0,0,0.091667


In [43]:
data.isna().sum()

age                       0
marie                     0
genre                     0
nb_jours_abonne           0
active_msg_vocaux         0
nb_msg_vocaux             0
nb_appel_jour             0
duree_appel_jour          0
cout_appel_jour           0
nb_appel_soiree           0
duree_appel_soiree        0
cout_appel_soiree         0
nb_appel_nuit             0
duree_appel_nuit          0
cout_appel_nuit           0
nb_appel_inter            0
duree_appel_inter         0
cout_appel_inter          0
nb_reclamation            0
churn                     0
offer_type_target_mean    0
dtype: int64

In [44]:
#Extraction des variables prédictives et cible
x=data.drop(["churn"], axis =1)
y=data["churn"]
x

,age,marie,genre,nb_jours_abonne,active_msg_vocaux,nb_msg_vocaux,nb_appel_jour,duree_appel_jour,cout_appel_jour,nb_appel_soiree,duree_appel_soiree,cout_appel_soiree,nb_appel_nuit,duree_appel_nuit,cout_appel_nuit,nb_appel_inter,duree_appel_inter,cout_appel_inter,nb_reclamation,offer_type_target_mean
0,19,0,0,34,0,0,80,128.8,21.90,93,208.7,17.74,103,202.1,9.09,7.0,14.0,3.78,1,0.144750
1,19,0,0,38,0,0,110,96.9,16.47,75,241.4,20.52,69,141.0,6.35,3.0,10.8,2.92,0,0.144750
2,19,0,0,43,1,36,68,93.1,15.83,83,246.1,20.92,109,94.9,4.27,8.0,8.7,2.35,1,0.144750
3,19,0,0,48,1,37,81,138.3,23.51,74,187.6,15.95,113,158.0,7.11,1.0,11.6,3.13,2,0.144750
4,19,0,0,60,0,0,77,207.3,35.24,105,207.9,17.67,89,108.2,4.87,5.0,12.9,3.48,1,0.144750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,80,1,1,71,0,0,130,123.8,21.05,91,226.5,19.25,96,213.9,9.63,7.0,8.8,2.38,2,0.144750
4996,80,1,1,82,0,0,112,172.4,29.31,89,131.7,11.19,78,182.8,8.23,9.0,8.8,2.38,1,0.144750
4997,80,1,1,137,0,0,107,75.8,12.89,98,256.9,21.84,118,204.4,9.20,3.0,10.5,2.84,2,0.144750
4998,80,1,1,139,1,23,129,157.6,26.79,96,247.0,21.00,112,259.2,11.66,2.0,13.7,3.70,0,0.091667


In [45]:
np.random.seed(0)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=15, stratify=y)
print("Train set shape:", x_train.shape, y_train.shape)
print("Test set shape:", x_test.shape, y_test.shape)

Train set shape: (4000, 20) (4000,)
Test set shape: (1000, 20) (1000,)


In [46]:
#Standarisation et Normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [47]:
#RandomForest

#Creating Model
randomforest = RandomForestClassifier(n_estimators=250, max_depth=18)

#Fitting Model
randomforest.fit(x_train,y_train)



RandomForestClassifier(max_depth=18, n_estimators=250)

In [57]:


# Predicting training accuracy
pred_train = randomforest.predict(x_train)

# Printing training accuracy
accuracy_train = accuracy_score(y_train, pred_train)
print("Training Accuracy:", accuracy_train)

# Predicting testing accuracy
pred_test = randomforest.predict(x_test)

# Printing testing accuracy
accuracy_test = accuracy_score(y_test, pred_test)
print("Testing Accuracy:", accuracy_test)


Training Accuracy: 0.99975
Testing Accuracy: 0.926


In [58]:


# Grid Search to improve accuracy with cross-validation


# Define cross-validation strategy (e.g., using KFold with 5 folds)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search to improve accuracy with cross-validation
parameters = {'n_estimators': [150, 200, 250, 300], 'max_depth': [15, 20, 25]}
forest = RandomForestClassifier()
clf = GridSearchCV(estimator=forest, param_grid=parameters, n_jobs=-1, cv=cv)

# Fitting the model
clf.fit(x_train, y_train)

# Getting the best parameters and score
print("Best Parameters for Random Forest:")
print(clf.best_params_)
print("Best Score:", clf.best_score_)



Best Parameters for Random Forest:
{'max_depth': 20, 'n_estimators': 200}
Best Score: 0.9305


In [61]:
# Getting the best parameters and score
clf.best_params_
{'max_depth': 20, 'n_estimators': 200}
clf.best_score_


0.9305

In [62]:
# Train Final Model with Best Parameters
final_model = RandomForestClassifier(n_estimators=clf.best_params_['n_estimators'], 
                                      max_depth=clf.best_params_['max_depth'])
final_model.fit(x_train, y_train)

# Evaluate Final Model
# Assuming you have a separate validation or test dataset named x_val and y_val
pred_val = final_model.predict(x_test)

# Compute evaluation metrics
accuracy_val = accuracy_score(y_test, pred_val)
# You can compute other evaluation metrics as well

# Print evaluation metrics
print("Validation Accuracy:", accuracy_val)
# You can print other evaluation metrics here as well


Validation Accuracy: 0.924


In [66]:
# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, pred_test)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matrice de Confusion')
plt.xlabel('Classe Prédite')
plt.ylabel('Classe Actuelle')
plt.show()

In [68]:
# Compute precision-recall curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
precision, recall, _ = precision_recall_curve(y_test, final_model.predict_proba(x_test)[:, 1])
pr_auc = auc(recall, precision)

# Plot precision-recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'PR Curve (AUC = {pr_auc:.2f})', color='b')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')
plt.show()

In [70]:
# Compute ROC curve
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, final_model.predict_proba(x_test)[:, 1])

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve', color='b')
plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()


In [72]:
# Compute learning curve
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(final_model, x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5)

# Plot learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training Score')
plt.plot(train_sizes, np.mean(valid_scores, axis=1), label='Validation Score')
plt.xlabel('Training Examples')
plt.ylabel('Score')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()


In [77]:
#SVM classifier
# Grid Search To Get Best Hyperparameters
parameters = {'C':[0.01,0.1,1,3,5,10]}
svmclf = SVC(class_weight='balanced',random_state=43)
grid = GridSearchCV(estimator=svmclf, param_grid=parameters,scoring='accuracy',return_train_score=True,verbose=1)
grid.fit(x_train,y_train)




Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=SVC(class_weight='balanced', random_state=43),
             param_grid={'C': [0.01, 0.1, 1, 3, 5, 10]},
             return_train_score=True, scoring='accuracy', verbose=1)

In [78]:
# Plotting the values
cv_result = pd.DataFrame(grid.cv_results_)
plt.scatter(cv_result['param_C'],cv_result['mean_train_score'])
plt.plot(cv_result['param_C'],cv_result['mean_train_score'],label='Train')
plt.scatter(cv_result['param_C'],cv_result['mean_test_score'])
plt.plot(cv_result['param_C'],cv_result['mean_test_score'],label="CV")
plt.title('Hyperparameter vs accuracy')
plt.legend()
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.show()

In [75]:


# Training the model using the optimal parameters discovered with SVM Classifier
svmclf =  SVC(C=3,class_weight='balanced', random_state=43)
svmclf.fit(x_train,y_train)

result2 = ["2.","SVM","Balanced using class weights"]
y_pred_tr = svmclf.predict(x_train)
print('Train accuracy SVM: ',accuracy_score(y_train,y_pred_tr))
result2.append(round(accuracy_score(y_train,y_pred_tr),2))

y_pred_test = svmclf.predict(x_test)
print('Test accuracy SVM: ',accuracy_score(y_test,y_pred_test))
result2.append(round(accuracy_score(y_test,y_pred_test),2))

recall = recall_score(y_test,y_pred_test)
print("Recall Score: ",recall)
result2.append(round(recall,2))

# Building a confusion matrix
matrix = confusion_matrix(y_test,y_pred_test)
ax=plt.subplot();
sns.heatmap(matrix, annot=True, fmt='d', linewidths=2, linecolor='black', cmap='YlGnBu',ax=ax)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_ylim(2.0,0)
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['Non Churn','Churn'])
ax.yaxis.set_ticklabels(['Non Churn','Churn'])
plt.show()


Train accuracy SVM:  0.949
Test accuracy SVM:  0.868
Recall Score:  0.5563380281690141


In [76]:
# Train Final Model with Best Parameters
final_svm_model = SVC(kernel='rbf', C=grid.best_params_['C'], gamma='scale')  # Adjust parameters as per grid search results
final_svm_model.fit(x_train, y_train)

# Evaluate Final Model
# Assuming you have a separate validation or test dataset named x_test and y_test
pred_test = final_svm_model.predict(x_test)

# Compute evaluation metrics
accuracy_test = accuracy_score(y_test, pred_test)
# You can compute other evaluation metrics as well

# Print evaluation metrics
print("Testing Accuracy:", accuracy_test)
# You can print other evaluation metrics here as well


Testing Accuracy: 0.899
